In [132]:
%load_ext autoreload
%autoreload 2

c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
import os
api_key_openai = os.environ.get('OPENAI_API_KEY')
org_openai = os.environ.get('OPENAI_ORG')

import openai
openai.organization = org_openai

In [134]:
import os
import sys
import inspect

sys.path.insert(0, '../..')

import pickle

In [135]:
from src.GetEmbeddings.GetEmbeddingsOpenAI import GetEmbeddingsOpenAI

In [136]:
from src.Clustering.Clustering import Clustering_and_DimRed

In [137]:
data = pickle.load(open("../../Data/Emebeddings/embeddings_reuters_raw.pkl", "rb"))
embeddings = data["embeddings"]
corpus = data["corpus"]

In [138]:
clusterer = Clustering_and_DimRed()
dim_red_embeddings, clusters = clusterer.cluster_and_reduce(embeddings)

UMAP(angular_rp_forest=True, metric='cosine', min_dist=0, n_components=5, random_state=42, verbose=True)
Sat Jul 29 09:52:23 2023 Construct fuzzy simplicial set
Sat Jul 29 09:52:23 2023 Finding Nearest Neighbors
Sat Jul 29 09:52:23 2023 Building RP forest with 10 trees
Sat Jul 29 09:52:23 2023 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	 6  /  13
	Stopping threshold met -- exiting after 6 iterations
Sat Jul 29 09:52:24 2023 Finished Nearest Neighbor Search
Sat Jul 29 09:52:24 2023 Construct embedding


Epochs completed: 100%| ██████████ 200/200 [00:06]


Sat Jul 29 09:52:31 2023 Finished embedding


In [139]:
import ExtractTopWords

In [140]:
extractor = ExtractTopWords.ExtractTopWords()

In [141]:
centroid_dict = extractor.extract_centroids(dim_red_embeddings, clusters)

In [142]:
extractor.compute_centroid_similarity(dim_red_embeddings, extractor.extract_centroids(dim_red_embeddings, clusters), 0)

array([0.00458414, 0.00517667, 0.00517025, ..., 0.00498587, 0.00548446,
       0.00513136], dtype=float32)

In [143]:
vocab = extractor.compute_corpus_vocab(corpus)

Processing corpus: 100%|██████████| 10788/10788 [00:10<00:00, 1006.06it/s]


In [144]:
len(vocab)

12328

In [145]:
words_per_topic = extractor.compute_words_topics(corpus, vocab, clusters)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Computing words per topic: 100%|██████████| 10788/10788 [00:06<00:00, 1787.31it/s]


In [146]:
with open("..\\..\\Data\Emebeddings\\vocab_reuters_embeddings.pkl", "rb") as f:
    embeddings_vocab = pickle.load(f)

In [156]:
vocab = list(embeddings_vocab.keys())

In [158]:
embeddings_vocab

{"'protectionism": array([-0.02040513, -0.02742023,  0.01220974, ...,  0.00547804,
        -0.02509964, -0.01084939]),
 "'the": array([-0.01447807,  0.01398942, -0.00478181, ..., -0.02067698,
         0.00279055, -0.02560538]),
 "'triple": array([-0.03341294,  0.0097142 ,  0.0111307 , ..., -0.00924897,
        -0.00981141, -0.02519164]),
 "'we": array([ 0.00537589,  0.00428523,  0.01002702, ..., -0.01336232,
        -0.00720186, -0.00173186]),
 '-dealers': array([-0.0193319 , -0.00155163,  0.00282889, ..., -0.00699384,
        -0.01089975, -0.01561003]),
 '-led': array([-0.00622114, -0.00846948,  0.0067523 , ..., -0.01509081,
        -0.00842583, -0.00714522]),
 '-official': array([-0.01431551, -0.00893727, -0.00185228, ..., -0.00580823,
        -0.00337876, -0.01006187]),
 '-stoltenberg': array([-0.01007332, -0.0174982 , -0.0072997 , ...,  0.00362679,
         0.01270201, -0.0072997 ]),
 '-trade': array([-0.00573205, -0.02175866,  0.02628718, ..., -0.01642779,
        -0.01262682, -0.

In [147]:
tfidf, tws = extractor.extract_topics_tfidf(corpus, vocab, clusters, top_n_words=10)

Computing word-topic matrix: 100%|██████████| 10788/10788 [01:21<00:00, 131.94it/s]

(12328, 118) (12328,)



c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src\ExtractTopWords\ExtractTopWords.py:222: RuntimeWarning: invalid value encountered in divide
  labels: np.ndarray, cluster labels. -1 means outlier
c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src\ExtractTopWords\ExtractTopWords.py:223: RuntimeWarning: divide by zero encountered in divide
  top_n_words: int, number of top words to extract per topic
c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src\ExtractTopWords\ExtractTopWords.py:227: RuntimeWarning: invalid value encountered in multiply
  # compute for each cluster how often each word occurs in the cluster


In [148]:
tfidf[12]

['enrollment',
 'dmp.mo',
 'baked',
 'bakeries',
 'kodak',
 'kock',
 'bally',
 'mathews',
 'matra',
 'aegean']

In [151]:
extractor.get_most_similar_docs(corpus, dim_red_embeddings, clusters, centroid_dict, cluster_label = 12, top_n = 3)

['CITIBANK LOWERS AUSTRALIAN PRIME RATE TO 16 PCT   Citibank Ltd said it would lower its   Australian prime rate to 16 pct from 16.5, effective tomorrow.       The new rate, if unmatched by other banks, will be the   lowest among Australian trading banks. Other primes range from   16.25 to 17.5 pct.       Australian prime rates have now retreated from a recent   peak of 19 pct in October in line with declining money market   levels. Citibank said its reduction reflected the decline.     ',
 'COMMONWEALTH BANK CUTS AUSTRALIAN SPLIT PRIME   The Commonwealth Bank of Australia said   it will lower its reference rate for loans to 15.75 pct from   16.25 pct and its overdraft reference rate to 16.25 pct from   16.75, effective June 24.       Bank officials have said the bank regards the overdraft   reference rate, based on short-term rate trends, as its key   prime lending rate to corporate customers. The loan reference   rate is based on longer term trends.       The bank is the latest to cu

In [162]:
centroid_dict_org_space = extractor.extract_centroids(embeddings, clusters)

In [163]:
tws = extractor.extract_top_words_centroid_similarity(vocab, embeddings_vocab, clusters, centroid_dict_org_space, 10)

In [173]:
tws[9]

['-trade',
 '-dealers',
 '-official',
 '1-apr-1987',
 '-led',
 "'protectionism",
 "'the",
 "'we",
 "'triple",
 '-stoltenberg']